In [40]:
class Note:
    
    def __init__(self, channel, pitch, velocity, time):
        self.channel = channel
        self.pitch = pitch
        self.velocity = velocity
        self.time = time
        
    def __eq__(self, other):
        return self.channel == other.channel and self.pitch == other.pitch and self.velocity == other.velocity and self.time == other.time
        
    def __repr__(self):
        return f"[Note {sself.channel} {self.pitch} {s} {}]"
        
    def __hash__(self):
        return hash(str(self))

In [42]:
from mido import Message, MidiFile, MidiTrack

mid = MidiFile('the_real_folk_blues.mid')

output_mid = MidiFile()
output_track = MidiTrack()
output_mid.tracks.append(output_track)

for i, track in enumerate(mid.tracks):
    print(f'Track {i}, {track.name}')
    
    for msg in track:
        # print(msg)
        output_track.append(msg)
    
print('finished')
output_mid.save('generated.mid')

note_set = set()
counter = 0

for msg in output_mid.tracks[0]:
    
    if msg.type == 'note_on':
        # print(msg)
        
        note = Note(msg.channel, msg.note, msg.velocity, msg.time)
        # note_set.add((msg.note, msg.velocity, msg.time, msg.channel))
        note_set.add(note)
        counter += 1
        
print(len(note_set), counter)

Track 0, uTHE REAL FOLK BLUES `sAm@[W`v                  
finished
4200 4200
